# HNLQ: Part-01

In this series of notebooks, we will demonstrate **hierarchical nested lattice quantization** using the D4 lattice, showing how multi-level quantization can improve rate-distortion performance.

In Part-01, we primarily check the API for creating the quantizer object, and see it side-by-side by with NLQ.

In [1]:
import sys
sys.path.append('../../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gemvq.quantizers.hnlq import HNLQ as Quantizer
from gemvq.quantizers.hnlq import HNLQConfig as QuantizerConfig

%load_ext autoreload
%autoreload 2


## Object Instantiation

An important observation. In terms API and implementation HNLQ and NLQ are very similar. In fact, NLQ is special case of HNLQ with M=1.  Therefore, whatever we observed for NLQ, we should observe for NLQ with M=1.

The NLQ API is a strict subset of HNLQ API. That is what we will see below, as sanity check.

### via Config

In [2]:
# create a quantizer for D4 lattice using QuantizerConfig object

# Method 1: Using QuantizerConfig object (most explicit)
config = QuantizerConfig(
    lattice_type='D4',      # Automatically loads D4 lattice components
    q=2,                    # Quantization parameter (alphabet size)
    beta=1.0,              # Scaling parameter for quantization
    alpha=0.5,             # Scaling parameter for overload handling
    eps=1e-8,              # Small perturbation parameter
    overload=True,          # Handle overload by scaling
    max_scaling_iterations=10,
    with_tie_dither=True,   # Add dither for tie breaking
    with_dither=False,       # No randomized dither    ,
    M =1,
    decoding='full'
)

quantizer = Quantizer(config)

def print_quantizer_summary(quantizer):
    d = len(quantizer.G)
    x = np.random.randn(d)
    x = x/np.sqrt(np.linalg.norm(x))
    print("Original vector:", x)


    x_hat = quantizer.quantize(x)
    

    enc, T = quantizer.encode(x)
    x_dq = quantizer.decode(enc, T)

    
    print(f"Lattice type: {quantizer.lattice_type}")
    print(f"Input: {x}")
    print(f"Encoded: {enc}")
    print(f"Scaling iterations: {T}")
    print(f"Reconstructed: {x_hat}")
    print(f"Error: {np.mean((x - x_hat)**2):.6f}")


print_quantizer_summary(quantizer)

Original vector: [ 0.80024382 -0.57503248  0.43420994 -0.66393914]
Lattice type: D4
Input: [ 0.80024382 -0.57503248  0.43420994 -0.66393914]
Encoded: (array([0, 1, 0, 1]),)
Scaling iterations: 0
Reconstructed: [ 1. -1.  1. -1.]
Error: 0.163389


###  via Dictionary

In [3]:
# Create configuration as dictionary
config_dict = {
    'lattice_type': 'D4',
    'q': 3,
    'beta': 1.0,
    'alpha': 1.0,
    'eps': 1e-8,
    'M': 3,
    'overload': True,
    'decoding': 'coarse_to_fine',
    'max_scaling_iterations': 10,
    'with_tie_dither': True,
    'with_dither': False
}

# Create HNLQ with configuration dictionary
quantizer = Quantizer(config_dict)
print_quantizer_summary(quantizer)


Original vector: [-0.49306207  0.0034824   1.55767593  0.46000833]
Lattice type: D4
Input: [-0.49306207  0.0034824   1.55767593  0.46000833]
Encoded: (array([2, 2, 1, 0]), array([0, 0, 0, 0]), array([0, 0, 0, 0]))
Scaling iterations: 0
Reconstructed: [0. 0. 2. 0.]
Error: 0.162595


### via specialized constructors

In [4]:
# Create Z² hierarchical quantizer
quantizer = Quantizer.create_z2_quantizer(
    q=3,
    M=3,
    beta=1.0,
    alpha=1.0,
    eps=1e-8,
    overload=True,
    decoding='full',
    with_tie_dither=True,
    with_dither=False
)
print_quantizer_summary(quantizer)

Original vector: [ 0.97886112 -1.40825796]
Lattice type: Z2
Input: [ 0.97886112 -1.40825796]
Encoded: (array([1, 2]), array([0, 0]), array([0, 0]))
Scaling iterations: 0
Reconstructed: [ 1. -1.]
Error: 0.083561


### via Custom Lattice

In [5]:
# Define custom generator matrix and closest point function
G_custom = np.array([[1, 0], [0, 1]])  # 2x2 identity matrix
def custom_closest_point(x):
    return np.floor(x + 0.5)

# Create configuration
config = QuantizerConfig(
    lattice_type='Z2',  # Can be any supported type
    q=3,
    M=2,
    beta=1.0,
    alpha=1.0,
    eps=1e-8
)

# Create HNLQ with custom components
quantizer = Quantizer(config, G=G_custom, Q_nn=custom_closest_point)
print_quantizer_summary(quantizer)

Original vector: [-1.29384757 -0.49603486]
Lattice type: Z2
Input: [-1.29384757 -0.49603486]
Encoded: (array([2, 0]), array([0, 0]))
Scaling iterations: 0
Reconstructed: [-1.  0.]
Error: 0.166198


### with Minimal Configuration

In [6]:
# Minimal configuration with defaults
config_minimal = QuantizerConfig(
    lattice_type='D4',
    q=3,
    M=3
    # All other parameters use defaults
)
quantizer = Quantizer(config_minimal)
print_quantizer_summary(quantizer)


Original vector: [ 0.09370087 -0.51599423 -0.04602902  0.9699695 ]
Lattice type: D4
Input: [ 0.09370087 -0.51599423 -0.04602902  0.9699695 ]
Encoded: (array([0, 0, 2, 2]), array([0, 0, 0, 0]), array([0, 0, 0, 0]))
Scaling iterations: 0
Reconstructed: [ 0. -1.  0.  1.]
Error: 0.061515


### Different decoding strategies

In [7]:
# Full decoding (default)
hnlq_full = Quantizer.create_d4_quantizer(q=3, M=3, decoding='full')

# Coarse-to-fine decoding
hnlq_coarse = Quantizer.create_d4_quantizer(q=3, M=3, decoding='coarse_to_fine')

# Progressive decoding
hnlq_progressive = Quantizer.create_d4_quantizer(q=3, M=3, decoding='progressive')

### Advanced configurations

In [8]:
# Advanced configuration with all parameters specified
config_advanced = QuantizerConfig(
    lattice_type='E8',
    q=4,
    beta=0.5,
    alpha=1.5,
    eps=1e-10,
    M=4,
    overload=False,
    decoding='progressive',
    max_scaling_iterations=20,
    with_tie_dither=False,
    with_dither=True
)

hnlq_advanced = Quantizer(config_advanced)

## A simple example
of using hierarchical lattice quantization to quantize a 4D vector.

We will take $x=(1.2,-0.7,2.4,0.3)$

In [9]:
np.random.seed(31)

q = 2
beta = 1
alpha = 1
eps = 1e-8
M = 1
lattice_type = 'D4'
config = QuantizerConfig(lattice_type=lattice_type, q=q, beta=beta, alpha=alpha, eps=eps, M=M)
quantizer = Quantizer(config)
    
    

# Create test sample and normalize it
test_sample = np.array([1.2, -0.7, 2.4, 0.3])
test_sample = test_sample/np.linalg.norm(test_sample)  # normalize to unit vector
print("Original vector:", test_sample)

# Test quantization
x_q = quantizer.quantize(test_sample, with_dither=False)
print("Quantized vector:", x_q)

# Test encode/decode separately
enc, T = quantizer.encode(test_sample, with_dither=False)
x_dq = quantizer.decode(enc, T, with_dither=False)
print("encoded:", enc)
print("overloading factor:", T)
print("Decoded vector:", x_dq)

# Print error
error = np.linalg.norm(test_sample - x_dq)
print(f"Reconstruction Error (L2 norm): {error:.6f}")

Original vector: [ 0.43022084 -0.25096216  0.86044169  0.10755521]
Quantized vector: [1. 0. 1. 0.]
encoded: (array([1, 0, 1, 0]),)
overloading factor: 0
Decoded vector: [1. 0. 1. 0.]
Reconstruction Error (L2 norm): 0.647051
